In [1]:
import numpy as np
# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt
import random
# plt.style.use('seaborn-poster')

In [2]:
def In(num, a, length):
    for i in range(length):
        if num == a[i]:
            return True
    return False

In [3]:
def Count(infilepath, part_pdg):
    with open(infilepath, 'r') as Input:
        lines = Input.readlines()
    events = 0
    for line in lines:
        if line.startswith('P'):
            pdata = line.split()
#             print(pdata[2])
            if In(pdata[2], part_pdg, len(part_pdg)):
                events += 1
    return events

In [7]:
##Read the file and construct the tree
# dtype = [('pdgid', int), ('px', float), ('py', float),
#          ('pz', float), ('energy', float), ('mass', float), ('vertex', int)]
def ReadFile(infilepath, outfilepath1, outfilepath2, outfilepath3, experiments, charge_dic):
    with open(infilepath, 'r') as Input:
        lines = Input.readlines()
    Train = open(outfilepath1, 'w')
    Val = open(outfilepath2, 'w')
    Test = open(outfilepath3, 'w')
    
    charge = 0
    cur_ver = []
    cur_exp = {}
    first_exp = True
#     first_ver = True
    vertex = 0
    Bparts = []
    i = 1
    Flag = 0
    for line in lines:
        if line.startswith('E'):
            if vertex != 0:
                cur_exp[vertex] = cur_ver
#                 print(cur_ver)
                experiments.append(cur_exp)
                cur_exp = {}
#                 for exp in experiments:
#                     for pars in exp.values():
#                         for par in pars:
#                             print(par)
                ##need to clean the data of the Bparts
                Bparts = []
#                 mBparts = []
                vertex = 0
        elif line.startswith('V'):
            if vertex != 0:
                cur_exp[vertex] = cur_ver 
            cur_ver = []
            vertex += 1
        elif line.startswith('P'):
            pdata = line.split()
            cur_par = [int(pdata[2]), float(pdata[3]),
                               float(pdata[4]), float(pdata[5]),
                               float(pdata[6]), float(pdata[7]), 
                               int(pdata[11]), vertex]
            ##The second last one would be which vertex do particle decay 
            cur_ver.append(cur_par)
            ##when detecting 521 particles
            if int(pdata[2]) == 521:
                #meeting next 521 particle in different experiments
#                 print(charge)
#                 charge = 0
                Bparts = []
                ran = random.randint(1, 10)
        
                energy = cur_par[4]
                mass = cur_par[5]
                px = cur_par[1]
                py = cur_par[2]
                pz = cur_par[3]
                if ran == 1:
                    Flag = 0##This is for Test
                elif ran == 2:
                    Flag = 1##This is for Validation
                else:
                    Flag = 2##This is for Train
                if Flag == 2:
#                     print('E ', mass, ' ', i, file = Test)
                    print('E ',px, ' ' ,py, ' ', pz, ' ', energy, ' ', mass, ' ', i, file = Train)
                elif Flag == 1:
                    print('E ',px, ' ' ,py, ' ', pz, ' ', energy, ' ', mass, ' ', i, file = Val)
                else:
                    print('E',px, ' ' ,py, ' ', pz, ' ', energy, ' ', mass, ' ', i, file = Test)
                Bparts.append(int(pdata[11]))
                i = i + 1
#                 for j in cur_par:
#                     print(j, end = ' ', file = Val)
#                 print('', file = Val)
#                 print(cur_par, file = Output)
            if In(-cur_par[7], Bparts, len(Bparts)):
                Bparts.append(int(pdata[11]))
                if (cur_par[6] == 0):
                    ch = charge_dic.get(cur_par[0], 0)
                    if Flag == 2:
                        print(ch, end = ' ', file = Train)
                    elif Flag == 1:
                        print(ch, end = ' ', file = Val)
                    else:
                        print(ch, end = ' ', file = Test)
                    for k in cur_par:
                        if Flag == 2:
                            print(k, end = ' ', file = Train)
                        elif Flag == 1:
                            print(k, end = ' ', file = Val)
                        else:
                            print(k, end = ' ', file = Test)
                    if Flag == 2:
                        print('', file = Train)
                    elif Flag == 1:
                        print('', file = Val)
                    else:
                        print('', file = Test)
#                     if (cur_par[0]<0):
#                         charge = charge - 1
#                     else:
#                         charge = charge + 1
        if line == 'HepMC::IO_GenEvent-END_EVENT_LISTING':
            cur_exp[vertex] = cur_ver
            experiments.append(cur_exp)
            break
    Input.close()
    Train.close()
    Val.close()
    Test.close()

In [8]:
# #electron: +11 (charge=-1)
# position: -11 (charge=+1)
# muon: +13 (charge=-1)
# anti-muon: -13 (charge=+1)
# K+: +321 (charge=+1)
# K-: -321 (charge=-1)
# pi+: +211 (charge=+1)
# pi-: -211 (charge=-1)
# proton: 2212 (charge=+1)
# anti-proton: -2212 (charge=-1)
charge_dic = {}
charge_dic[11] = -1
charge_dic[-11] = 1
charge_dic[13] = -1
charge_dic[-13] = 1
charge_dic[321] = 1
charge_dic[-321] = -1
charge_dic[211] = 1
charge_dic[-211] = -1
charge_dic[2212] = 1
charge_dic[-2212] = -1
experiments = []
ReadFile('hepMCtest', 'train.txt', 'val.txt', 'test.txt', experiments, charge_dic)

In [6]:
def PrepareTest(inputfile, outputfile):
    with open(inputfile, 'r') as Input:
        lines = Input.readlines()
    Test = open(outputfile, 'w')
    for line in lines:
        if line.startswith('E'):
            ran = random.randint(1, 10)
            print('E ',ran, ' ' ,ran, ' ', ran, ' ', ran, ' ', ran, ' ', ran, file = Test)
        else:
            print(line, end = '', file = Test)

In [7]:
PrepareTest('val.txt', 'test.txt')

In [6]:
ls = [1,2,3,4]
n = 5
print(In(n, ls, len(ls)))

False


In [7]:
def Readpxpypz(filepath, px, py, pz):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    cur_exp = []
    experiments = []
    first = True

    for line in lines:
        if line.startswith('E'):
            if first and not cur_exp:
                first = False
            else:
                experiments.append(cur_exp)
                cur_exp = []
        elif line.startswith('P'):
            pdata = line.split()
            cur_par = [int(pdata[2]), float(pdata[3]),
                               float(pdata[4]), float(pdata[5]),
                               float(pdata[6]), float(pdata[6]),
                               float(pdata[7]), int(pdata[11])]
            if cur_par[7] == 0:
                px.append(cur_par[1]+20)
                py.append(cur_par[2]+20)
                pz.append(cur_par[3]+20)
            cur_exp.append(cur_par)
        if line == 'HepMC::IO_GenEvent-END_EVENT_LISTING':
            experiments.append(cur_exp)
            break